In [1]:
# import multiprocessing
from pyomo.environ import *
from pyomo.opt import SolverFactory
import math
# import cmath
import numpy as np
# from numpy import r_, c_, ix_, zeros, pi, ones, exp, argmax

import pandapower as pp
# from pandapower.pypower.makeYbus import makeYbus

from pandapower.pypower.makeBdc import makeBdc

# from pyomo.contrib.pynumero.interfaces.pyomo_nlp import PyomoNLP
# from pyomo.util.infeasible import log_infeasible_constraints
# import logging
from pandapower.pypower.idx_gen import PG, GEN_BUS, PMAX, PMIN
# from pandapower.pypower.idx_cost import NCOST, COST
from pandapower.pypower.idx_brch import PF, PT, QF, QT, RATE_A, F_BUS, T_BUS
from pandapower.pypower.idx_bus import PD

In [2]:
path = 'C:\\Users\\RichriD\\Downloads\\Compressed\\Ipopt-3.14.17-win64-msvs2022-md\\bin\\ipopt.exe'

# 导入参数

In [3]:
# get the structured data from pypower
net = pp.networks.case9()
pp.runpp(net)
ppc = pp.converter.to_ppc(net)

baseMVA, bus, gen, branch = ppc["baseMVA"], ppc["bus"], ppc["gen"], ppc["branch"]

Bbus, _, _, _, _ = makeBdc(bus, branch)
B = Bbus.todok()
B = dict(B.items())

# 获取 from_bus 和 to_bus，直接转换为整数数组
from_bus = branch[:, 0].astype(int)
to_bus = branch[:, 1].astype(int)

# 直接使用 zip() 组合为 (from_bus, to_bus) 形式，并转换为列表
line_list = list(zip(from_bus, to_bus))

# generate bus-gen_matrix for parameters initializing
row_bus, column_bus = bus.shape
row_gen, column_gen = gen.shape

pg_max_list = np.zeros(row_bus)
pg_min_list = np.zeros(row_bus)
pg_list = np.zeros(row_bus)

# 获取所有发电机所在的节点索引
gen_bus = gen[:, GEN_BUS].astype(int)

pg_max_list[gen_bus] = gen[:, PMAX] / baseMVA
pg_min_list[gen_bus] = gen[:, PMIN] / baseMVA
pg_list[gen_bus] = gen[:, PG] / baseMVA

type_list = bus[:,1]

# 获取发电机成本矩阵
gencost_mtrx = np.array(ppc['gencost'])  # 确保是 NumPy 数组
row_cost, column_cost = gencost_mtrx.shape  # 获取形状

# 初始化 bus-gen 成本矩阵, 一般都是包含常数项的2次函数
bus_gen_mtrx = np.zeros((row_bus, 3))

bus_gen_mtrx[gen_bus, :] = gencost_mtrx[:, -3:]
# transefer the matrix to dic
cost_dic = {(i, j): value 
            for i, row in enumerate(bus_gen_mtrx) 
            for j, value in enumerate(row)}

# 定义发电机的碳排放强度
ei = np.zeros(row_bus) * 0.3
ei[gen_bus] = [0.3, 0.5, 0.8]

# 获取支路输电容量限制
PLMAX = branch[:, RATE_A] / baseMVA

# 默认设置中最大负荷为 1.25 pu, 将排放值限制在暂时这么设置
Emis_cap = np.ones(row_bus) * 0.8

In [4]:
# bus[6, PD] += 1

In [5]:
model = ConcreteModel()

## 定义 Set

In [6]:
# Define the sets
model.buses = Set(initialize = bus[:,0].astype(int))
model.lines = Set(initialize = range(len(line_list)))
model.cost_dims = Set(initialize = range(0, 3))

## 定义 Param

In [7]:
# Define the parameters (change the bus-> dict)
model.PD = Param(model.buses, initialize = bus[:, PD]/baseMVA, default = 0.0) # default meaning?

model.PG_MAX = Param(model.buses, initialize = pg_max_list, default = 0.0)
model.PG_MIN = Param(model.buses, initialize = pg_min_list, default = 0.0)

model.B = Param(model.buses*model.buses, initialize = B, default = 0.0)

model.C = Param(model.buses*model.cost_dims, initialize = cost_dic, default = 0.0)

model.PL_MAX = Param(model.lines, initialize = PLMAX, default = 0.0)

# emissions cap
model.Emis_cap = Param(model.buses, initialize = Emis_cap, default = 0.0)

# 这种方法目前只针对 IEEE 9-BUS 生效, 因为所有发电机都是接在一个单项连通电网节点中, 可以保证该节点为发电机占优
model.w_g = Param(model.buses, initialize = ei, default = 0.0)

In [8]:
# model.PD.display()

In [9]:
# model.PD[6] += 0.01

In [10]:
# model.PD.display()

## 定义 Var

In [11]:
# Define the variables
model.PG = Var(model.buses, domain = Reals)
model.theta = Var(model.buses, initialize=0, bounds=(-math.pi/8, math.pi/8))

# node cei
model.w = Var(model.buses, initialize=ei, bounds=(0, 0.8))

In [12]:
# 关于表示潮流流向的变量
model.p_out = Var(model.buses, model.buses, initialize=0, domain=NonNegativeReals)  # i -> j 在 i 处的量测
model.p_in = Var(model.buses, model.buses, initialize=0, domain=NonNegativeReals)  # j -> i 在 i 处的量测

In [13]:
for i in model.buses:
    model.p_out[i, i].fix(0)
    model.p_in[i, i].fix(0)

In [14]:
for i in model.buses:
    for j in set(np.where(Bbus.toarray()[i, :] == 0)[0]):
        model.p_out[i, j].fix(0)
        model.p_in[i, j].fix(0)

In [15]:
# Bbus.toarray()

In [16]:
# set(np.where(Bbus.toarray()[0, :] == 0)[0])

In [17]:
# for i in np.where(Bbus.toarray()[0, :] == 0):

In [18]:
# 根据节点类型定义发电机限额, 以及固定平衡节点的电压相角
for i, bus_type in enumerate(type_list):
    if bus_type in {2, 3}:
        model.PG[i].lb = pg_min_list[i]
        model.PG[i].ub = pg_max_list[i]
        model.w[i].fix(ei[i])

        if type_list[i] == 3:
            model.theta[i].fix(0)

    else:
        model.PG[i].fix(0)

## 定义 Constrain

In [19]:
# 双向潮流变量的自然约束
def branch_flow_direction(model, i, j):
    return model.p_out[i, j] * model.p_in[i, j] <= 0
# pij_i * pji_i == 0, 松弛为  pij_i * pji_i <= 0
model.branch_flow_direction = Constraint(model.buses, model.buses, 
                                         rule = branch_flow_direction)

In [20]:
# 双向潮流等效方法, 以及支路潮流输电限额
# pij_i - pji_i == pf_ij == 1/xij * (theta_i - theta_j)
# pij_j - pji_j == -1/xij * (theta_i - theta_j)
# 由于网络是无损的,其实就等效于 pij_i == pij_j
model.forward_branch_flow = ConstraintList()
model.backward_branch_flow = ConstraintList()
model.power_branch_flow = ConstraintList()  # --> \mu
for l in model.lines:
    fbus = branch[l, F_BUS]  # i
    tbus = branch[l, T_BUS]  # j
    B_ft = -model.B[fbus, tbus]
    
    pf_ij = B_ft * (model.theta[fbus] - model.theta[tbus])
    
    model.power_branch_flow.add(expr = abs(pf_ij) <= model.PL_MAX[l])
    
    model.forward_branch_flow.add(
        expr = model.p_out[fbus, tbus] - model.p_in[fbus, tbus] == pf_ij
    )
    model.backward_branch_flow.add(
        expr = model.p_in[tbus, fbus] - model.p_out[tbus, fbus] == pf_ij
    )

In [21]:
# Define the power balance constraints
model.power_balance_constraints = ConstraintList()
# 功率平衡 (DCPF)
for i in model.buses:
    # real_power_out = model.p_out[i] - model.p_in[i]
    real_power_out = sum(
        model.p_out[i, j] - model.p_in[i, j]
        for j in model.buses
    )
    
    if type_list[i] in {2, 3}:
        model.power_balance_constraints.add(expr = model.PG[i] - model.PD[i] == real_power_out)
    else:
        model.power_balance_constraints.add(expr = - model.PD[i] == real_power_out)

In [22]:
# power supply-demand
model.power_supply_demand_balance = Constraint(expr = sum(model.PG[i] - model.PD[i] for i in model.buses) == 0)

In [23]:
# 发电功率约束
model.generator_limits = ConstraintList()
for i in model.buses:
    if type_list[i] in {2, 3}:
        model.generator_limits.add(expr = model.PG_MIN[i] - model.PG[i] <= 0)
        model.generator_limits.add(expr = model.PG[i] - model.PG_MAX[i] <= 0)

In [24]:
# 碳流方程
model.carbon_emission_flow = ConstraintList()
for i in model.buses:
    p_inj = model.PG[i] + sum(model.p_in[i, j] for j in model.buses)
    emis_inj = model.w_g[i] * model.PG[i] + sum(model.p_in[i, j] * model.w[j] for j in model.buses)
    model.carbon_emission_flow.add(expr = p_inj * model.w[i] == emis_inj)

In [25]:
def carbon_cap(model, i):
    return model.w[i] * model.PD[i] <= model.Emis_cap[i]
model.carbon_cap = Constraint(model.buses, rule=carbon_cap)

# 定义目标函数

In [26]:
model.obj_cost = Objective(
    expr = sum(
        (model.PG[i] * baseMVA) ** 2 * model.C[i,0]+ (model.PG[i] * baseMVA) * model.C[i,1] + model.C[i,2]
        for i in model.buses
    ), 
    sense = minimize
)

In [27]:
model.dual = Suffix(direction=Suffix.IMPORT)

In [28]:
solver = SolverFactory('ipopt', executable=path)
solver.options['tol'] = 1e-6  # 允许更大的误差
solver.options['acceptable_tol'] = 1e-5 
results = solver.solve(model, tee = True)

Ipopt 3.14.17: tol=1e-06
acceptable_tol=1e-05


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      154
Number of nonzeros in inequality constraint Jacobian.:       62
Number of nonzeros in Lagrangian Hessian.............:       67

Total number of variables............................:       53
                     variables with only lower bounds:       36
                variables with lower and upper bounds:       17
                     variables with only upper bounds:        0
Total number of equality constraints.........

In [29]:
model.pprint()

3 Set Declarations
    buses : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {0, 1, 2, 3, 4, 5, 6, 7, 8}
    cost_dims : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    lines : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {0, 1, 2, 3, 4, 5, 6, 7, 8}

8 Param Declarations
    B : Size=81, Index=buses*buses, Domain=Any, Default=0.0, Mutable=False
        Key    : Value
        (0, 0) :  17.361111111111114
        (0, 3) : -17.361111111111114
        (1, 1) :                16.0
        (1, 7) :               -16.0
        (2, 2) :  17.064846416382252
        (2, 5) : -17.064846416382252
        (3, 0) : -17.361111111111114
        (3, 3) :   39.99538221085536
        (3, 4) : -10.869565217391305
        (3, 8) :  -11.76470588235294
        (4, 3

In [30]:
print(model.obj_cost())

5223.394300827324


In [31]:
for lam in model.power_supply_demand_balance:
    print(model.dual[model.power_supply_demand_balance[lam]])

2305.599476465665


In [32]:
M = len(branch)
N = len(bus)
A = np.zeros((M, N))
H = np.zeros((M, M))
for i in range(M):
    fbus = branch[i, F_BUS].astype(int)
    tbus = branch[i, T_BUS].astype(int)
    
    H[i, i] = - Bbus[fbus, tbus]
    A[i, fbus] = 1
    A[i, tbus] = -1

B_dash = np.delete(Bbus.toarray(), 0, axis=0)
B_dash = np.delete(B_dash, 0, axis=1)
B_dash_inv = np.linalg.inv(B_dash)
T = H @ A @ np.pad(B_dash_inv, ((1, 0), (1, 0)), mode='constant', constant_values=0)

In [33]:
for l in model.lines:
    fbus = branch[l, F_BUS]  # i
    tbus = branch[l, T_BUS]  # j
    B_ft = -model.B[fbus, tbus]
    
    pf_ij = B_ft * (model.theta[fbus].value - model.theta[tbus].value)
    print("fbus:{}, tbus:{}, pf:{}".format(fbus, tbus, np.round(pf_ij, 2)))

fbus:0.0, tbus:3.0, pf:0.86
fbus:3.0, tbus:4.0, pf:0.35
fbus:4.0, tbus:5.0, pf:-0.55
fbus:2.0, tbus:5.0, pf:0.88
fbus:5.0, tbus:6.0, pf:0.33
fbus:6.0, tbus:7.0, pf:-0.67
fbus:7.0, tbus:1.0, pf:-1.41
fbus:7.0, tbus:8.0, pf:0.74
fbus:8.0, tbus:3.0, pf:-0.51


In [34]:
epsilon = []
for i in model.carbon_emission_flow:
    epsilon.append(model.dual[model.carbon_emission_flow[i]])

In [45]:
model.dual[model.carbon_cap[6]]

-1100.7252874240496

In [46]:
T

array([[ 0.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00,
        -1.00000000e+00, -1.00000000e+00, -1.00000000e+00],
       [ 0.00000000e+00, -3.61339600e-01, -6.15158637e-01,
        -5.82210608e-17, -8.64864865e-01, -6.15158637e-01,
        -4.67097532e-01, -3.61339600e-01, -1.24853114e-01],
       [ 0.00000000e+00, -3.61339600e-01, -6.15158637e-01,
        -1.05162937e-16,  1.35135135e-01, -6.15158637e-01,
        -4.67097532e-01, -3.61339600e-01, -1.24853114e-01],
       [ 0.00000000e+00,  2.13887328e-16,  1.00000000e+00,
         1.08439613e-17,  8.86097128e-17,  1.97797137e-16,
        -2.60991512e-18,  2.13887328e-16,  0.00000000e+00],
       [ 0.00000000e+00, -3.61339600e-01,  3.84841363e-01,
         1.74193111e-16,  1.35135135e-01,  3.84841363e-01,
        -4.67097532e-01, -3.61339600e-01, -1.24853114e-01],
       [ 0.00000000e+00, -3.61339600e-01,  3.84841363e-01,
        -3.78709410e-19,  1.35135135e-01,  3.848413

In [47]:
clmp_6 = model.dual[model.power_supply_demand_balance]
p_in_i = np.array([model.p_in[6, i].value for i in model.buses])
upper_node_set = np.where(p_in_i>0)[0]

con_l = []
for l in model.lines:
    fbus = branch[l, F_BUS].astype(int) # i
    tbus = branch[l, T_BUS].astype(int)
    if fbus == 6 or tbus == 6:
        con_l.append(l)

lmp_carbon_1 = 0
lmp_carbon_2 = 0
for l in con_l:
    fbus = branch[l, F_BUS].astype(int)
    tbus = branch[l, T_BUS].astype(int)
    
    if fbus == 6 and tbus in set(upper_node_set):
        lmp_carbon_1 += T[l, 6] * model.w[tbus].value
        lmp_carbon_2 += T[l, 6]
    elif tbus == 6 and fbus in set(upper_node_set):
        lmp_carbon_1 += T[l, 6] * model.w[fbus].value
        lmp_carbon_2 += T[l, 6]
        
clmp_6 -= epsilon[6] * (lmp_carbon_1 - model.w[6].value*lmp_carbon_2)
clmp_6 += model.dual[model.carbon_cap[6]] * model.w[6].value

mu = []
for l in model.power_branch_flow:
    mu.append(model.dual[model.power_branch_flow[l]])

clmp_6 -= sum(
    mu[l] * T[l, 6]
    for l in model.lines
)

In [36]:
clmp_6

1483.677160403275

In [40]:
model.dual[model.carbon_cap[6]]

-1100.7252874240496

In [37]:
model.obj_cost()

5223.394300827324

In [38]:
model.w.display()

w : Size=9, Index=buses
    Key : Lower : Value               : Upper : Fixed : Stale : Domain
      0 :     0 :                 0.3 :   0.8 :  True :  True :  Reals
      1 :     0 :                 0.5 :   0.8 :  True :  True :  Reals
      2 :     0 :                 0.8 :   0.8 :  True :  True :  Reals
      3 :     0 :   0.300000004089916 :   0.8 : False : False :  Reals
      4 :     0 :  0.6056792274985369 :   0.8 : False : False :  Reals
      5 :     0 :  0.7999999914896562 :   0.8 : False : False :  Reals
      6 :     0 :  0.6000000094372914 :   0.8 : False : False :  Reals
      7 :     0 :  0.5000000002280421 :   0.8 : False : False :  Reals
      8 :     0 : 0.41837939533190804 :   0.8 : False : False :  Reals


In [42]:
for i in range(3):
    print(model.w_g[i] * model.PG[i].value)

0.2579718502872841
0.7032689147693679
0.7068448022303311


In [43]:
sum(model.w_g[i] * model.PG[i].value for i in model.buses)

1.6680855672869832

In [44]:
sum(model.w[i].value * model.PD[i] for i in model.buses)

1.6680855583508596

In [39]:
model.dual[model.power_supply_demand_balance]

2305.599476465665

In [39]:
1483.1819847492059, 5223.394297309541

(1483.1819847492059, 5223.394297309541)

In [40]:
5256.017563293774 - 5223.394297309541

32.62326598423351

In [48]:
model.carbon_cap.display()

carbon_cap : Size=9
    Key : Lower : Body               : Upper
      0 :  None :                0.0 :   0.6
      1 :  None :                0.0 :   0.6
      2 :  None :                0.0 :   0.6
      3 :  None :                0.0 :   0.6
      4 :  None : 0.5451113047486832 :   0.6
      5 :  None :                0.0 :   0.6
      6 :  None : 0.6000000094372914 :   0.6
      7 :  None :                0.0 :   0.6
      8 :  None :  0.522974244164885 :   0.6
